In [143]:
import glob
import os
import pathlib
import frontmatter
import re
import collections

import yaml


#------------------BEGIN HACKY --------------
_mapping_tag = yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG

def dict_representer(dumper, data):
    return dumper.represent_dict(data.items())

def dict_constructor(loader, node):
    loader.flatten_mapping(node)
    return collections.OrderedDict(loader.construct_pairs(node))

def null_representer(dumper, value):
    return dumper.represent_scalar(u'tag:yaml.org,2002:null', '')

try:
    yaml.CSafeDumper.add_representer(collections.OrderedDict, dict_representer)
    yaml.CSafeDumper.add_representer(type(None), null_representer)
except (AttributeError, ImportError):
    pass



yaml.SafeDumper.add_representer(collections.OrderedDict, dict_representer)
yaml.SafeDumper.add_representer(type(None), null_representer)
yaml.SafeLoader.add_constructor(_mapping_tag, dict_constructor)

#------------------END HACKY --------------


content = glob.glob('content/**/*.md')





In [144]:

for sourcefile in content:
    basename = os.path.basename(sourcefile)
    if basename == '_index.md':
        continue
    post = frontmatter.load(sourcefile)
    year = post.metadata['date'].year
    targetfolder = re.findall(r'^(\w+[/\\]\w+)',os.path.dirname(sourcefile))[0] 
    targetfolder = os.path.join(targetfolder,f'{year:.0f}')
    targetfile = os.path.join(targetfolder,basename)
    #print(targetfile,sourcefile)
    post.metadata = collections.OrderedDict(post.metadata)
    post.metadata.move_to_end('title', last=False)
    #then reverse
    
    os.makedirs(targetfolder, exist_ok=True)

    banner = post.metadata['banner']
    #images = re.findall(r'!\[[^\]]*\]\((.*?)\s*("(?:.*[^"])")?\s*\)',post.content) #https://stackoverflow.com/questions/44227270/regex-to-parse-image-link-in-markdown
    images = re.findall(r"([-/\w]+/[-/\w]+\.(?:jpg|gif|png|jpeg|webp|tif|tiff))",post.content) 
    images.append(banner)

    outputpost = frontmatter.dumps(post)
    for source_image_url in images:
        target_image = os.path.join(targetfolder, os.path.basename(source_image_url))
        target_image_url = target_image.replace('\\','/').replace('content','')
        if source_image_url[0]=='/':
            source_image = os.path.join('static',source_image_url[1:])
        else:
            print('NOT MOVING',source_image_url)
            continue
        #print('FILE:', source_image, '->', target_image)
        #print('URL:', source_image_url, '->', target_image_url)

        if not os.path.exists(source_image):
            print('path does not exist', source_image)
            continue

        outputpost = outputpost.replace(source_image_url,target_image_url)
        
        os.rename(source_image,target_image)

    os.remove(sourcefile)
    with open(targetfile, "w") as text_file:
        text_file.write(outputpost)



    
    #if len(images)>4:
    #    break

    #orig_bannerimg = 
    #print()
    #break

path does not exist static\2016/02/n55e012.png
path does not exist static\2016/02/bamberaspinall20s2-1.png
path does not exist static\2016/02/ICATdamagefrequency.png
path does not exist static\2016/02/projections.png
path does not exist static\2016/02/SBB20view.jpg
NOT MOVING com/035_Oligocene_3globes.jpg
path does not exist static\2008/02/1280px-Clouds_over_the_Atlantic_Ocean.jpg
path does not exist static\2008/02/smodel.jpg
path does not exist static\Home/PDFs/Announcements/Present-day-summers-in-Svalbard-are-warmer-than-the-medieval-warm-period-/washout.jpg
path does not exist static\2011/04/Genova-1810ca-acquatinta-Garneray-part-mura.jpg
path does not exist static\Home/PDFs/Announcements/commentonthesubsidenceadjustmentappliedtothekempetalproxyofnorthcarolinarelativesealevel/engelhart-kemp.png
path does not exist static\2016/02/moore2011.png
path does not exist static\2014/03/Pinatubo91eruption_plume.jpg
path does not exist static\2014/10/PR.png
path does not exist static\2016/02/P

<function OrderedDict.__reversed__>

TypeError: YAMLHandler.export() missing 1 required positional argument: 'metadata'

In [13]:
post = frontmatter.load(sourcefile)

In [18]:
post.metadata['banner']

'/2010/02/sia.jpg'

In [ ]:
re.findall(r"([-/\w]+/[-/\w]+\.(?:jpg|gif|png|jpeg|webp|tif|tiff))",'dsafdsaf asdfafds.png /123/432421/45_345.png dsfadf')

['23/432421/45_345.png']